# A summary of genotype sample LD in GTEx data

This is partially in response to the reviewer response for SuSiE paper.

Here I'd like to find out for a randomly chosen variant, how many others is it in LD >=0.8, 0.9,0.95 , 0.99 and 1.

In [ ]:
[global]
parameter: datadir = path('~/Documents/GTExV8/Toys/')
parameter: categories = [0.8,0.85,0.9,0.95,0.99, 1]
parameter: nvar = 100
parameter: workdir = path('~/GIT/LargeFiles/SuSiE_Revision_1/LD_Summary')

In [ ]:
[1]
# load all genotypes and computer LD
# For computational reasons let's not save them ... instead lets survey for each variant the different LD catagories
import glob
input: glob.glob(f"{datadir:a}/Thyroid.ENSG00000283440.RDS"), group_by = 1, concurrent = True
output: f"{workdir:a}/{_input:bn}.LD.rds"
task: trunk_workers = 1, trunk_size = 10, walltime = '2m', mem = '2G', cores = 1, tags = f'{step_name}_{_output:bn}'
R: workdir=workdir, expand = '${ }'
  set.seed(999)
  X = readRDS(${_input:r})$X
  muffled_corr = function(x)
  withCallingHandlers(cor(x),
                    warning = function(w) {
                      if (grepl("the standard deviation is zero", w$message))
                        invokeRestart("muffleWarning")
                    } )
  R = muffled_corr(X)
  # mask self-correlation out to zero
  diag(R) = 0
  var_id = sort(sample(1:nrow(R), size = min(${nvar}, nrow(R))))
  res = list()
  i = 1
  categories = c(${",".join(map(str,categories))})
  for (item in categories) {
      res[[i]] = rowSums(R[var_id, ] >= item)
      i = i + 1
  }
  res = do.call(cbind, res)
  colnames(res) = as.character(categories)
  saveRDS(res, ${_output:r})

In [ ]:
[2]
input: group_by = 'all'
output: f"{workdir:a}/LD_Summary.rds")
R: workdir = workdir, expand = '${ }'
  for (item in c(${_input:a,})) {
      if (is.null(res)) res = readRDS(item)
      else res = rbind(res, readRDS(item))
  }
  saveRDS(res, ${_output:r})